# Resolvendo o problema: Sliding Puzzle

<img src='https://github.com/alvaromfcunha-c210/aula5/blob/master/imagens/1.gif?raw=1' alt='1' width='300' height='300'>

### Criação do problema:

In [1]:
import numpy as np

class SlidingPuzzle:

  #Definindo a estrutura do meu sliding-puzzle
  #Ex: problema = SlidingPuzzle(3)
  #O parêmtro 3 significa que eu terei um puzzle 3x3
  def __init__(self, num_blocos):
    self.num_blocos = num_blocos

  #Busca a posição de um determinado elemento dentro de uma matriz estado.
  #Esse determinado elemento é o de valor 0
  #Se o elemento encontrado na matriz estado for 0, retornará a sua posição
  #OBS: Se encontrar o elemento, retorna a posição (i, j) correspondente; caso contrário, retorna (None, None).
  def _encontra_posicao(self, estado, elemento):
    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        if elemento == estado[i, j]:
          return i, j
    return None, None

  #Compara duas matrizes atual e objetivo para verificar se são idênticas.
  #Percorre as posições das matrizes e verifica se o valor em cada posição é o mesmo.
  def verifica_estados(self, atual, objetivo):
    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        if atual[i, j] != objetivo[i, j]:
          return False
    return True

  def expande_estados(self, atual):

    #Lista que armazenará os novos estados gerados ao mover a peça vazia nas diferentes direções (cima, baixo, esquerda e direita)
    # A cada novo estado gerado, ele é adicionado à lista novos_estados usando o método append.
    #Isso permite que a lista acumule todos os possíveis estados resultantes de um movimento válido da peça vazia.
    novos_estados = []

    #A função recebe uma matriz atual e calcula a posição da peça vazia (representada por 0) usando o método _encontra_posicao.
    linha, coluna = self._encontra_posicao(atual, 0)

    # Cima
    #Verifica se a peça vazia pode ser movida para cima (linha não é 0), cria um novo estado copiando a matriz atual
    #e trocando os valores da peça vazia e da peça acima dela.
    if linha != 0:
      nova_linha = linha - 1
      novo_estado = np.copy(atual)

      bloco_alvo = atual[nova_linha, coluna]
      novo_estado[nova_linha, coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Baixo
    #Move a peça vazia para baixo, desde que a posição da linha não seja a última.
    if linha != self.num_blocos - 1:
      nova_linha = linha + 1
      novo_estado = np.copy(atual)

      bloco_alvo = atual[nova_linha, coluna]
      novo_estado[nova_linha, coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Esquerda
    #Move a peça vazia para a esquerda, verificando se a coluna não é 0.
    if coluna != 0:
      nova_coluna = coluna - 1
      novo_estado = np.copy(atual)

      bloco_alvo = atual[linha, nova_coluna]
      novo_estado[linha, nova_coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Direita
    #Move a peça vazia para a direita, desde que a posição da coluna não seja a última.
    if coluna != self.num_blocos - 1:
      nova_coluna = coluna + 1
      novo_estado = np.copy(atual)

      bloco_alvo = atual[linha, nova_coluna]
      novo_estado[linha, nova_coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    return novos_estados

### Usando busca em largura (BFS):

<img src='https://github.com/alvaromfcunha-c210/aula5/blob/master/imagens/2.gif?raw=1' alt='2' width='300' height='300'>

In [2]:
from queue import Queue

class BreadthFirstSearch:
  def __init__(self, problema: SlidingPuzzle):
    self.problema = problema

  #Verifica se um determinado estado já foi visitado, comparando-o com os estados presentes na lista estados_visitados.
  #Compara se o estado atual é igual a algum dos estados visitados.
  #Se encontrar um estado igual, retorna True, indicando que o estado já foi visitado, caso contrário, retorna False.
  def _verifica_visitados(self, estado, estados_visitados):
    for i in estados_visitados:
      if self.problema.verifica_estados(i, estado):
        return True
    return False

  #Realizando a Busca em Largura
  def busca(self, inicio, fim):
    #Queue (fila) é uma estrutura de dados que segue o princípio "primeiro a entrar, primeiro a sair" (First-In-First-Out, ou FIFO).
    fila = Queue()
    fila.put(inicio)

    solucao_encontrada = False
    estados_visitados = []
    cont_estados = 0

    while not fila.empty():
      #Retira o estado mais antigo da fila usando fila.get().
      atual = fila.get()

      #Adiciona o estado atual à lista de estados_visitados.
      estados_visitados.append(atual)

      #Verifica se o estado atual é igual ao estado fim usando o método verifica_estados do problema.
      #Se for igual, define solucao_encontrada como True e interrompe o loop.
      if self.problema.verifica_estados(atual, fim):
        solucao_encontrada = True
        break
      #Caso contrário, incrementa cont_estados para contar quantos estados foram visitados.
      else:
        cont_estados += 1

        # print(f'Visitando estado: {cont_estados}')
        #Obtém os novos estados possíveis a partir do estado atual usando self.problema.expande_estados(atual).
        novos_estados = self.problema.expande_estados(atual)

        #Itera por esses novos estados e verifica se eles já foram visitados usando o método _verifica_visitados.
        #Se não foram visitados, coloca esses novos estados na fila.
        for i in novos_estados:
          if not self._verifica_visitados(i, estados_visitados):
            fila.put(i)

    return solucao_encontrada, estados_visitados, cont_estados

# **A busca em largura explora todos os estados possíveis em cada nível de profundidade antes de prosseguir para os níveis seguintes. Isso garante que a primeira solução encontrada é uma das mais curtas possíveis em termos de número de movimentos.**

In [3]:
problema = SlidingPuzzle(3)
bfs = BreadthFirstSearch(problema)

inicio = np.array([[2, 8, 3],
                    [1, 6, 4],
                    [7, 0, 5]])

alvo = np.array([[1, 2, 3],
                  [8, 0, 4],
                  [7, 6, 5]])

solucao, visitados, passos = bfs.busca(inicio, alvo)

print(f'Solução encontrada? {solucao}' + '\n',
      f'Estados visitados: {visitados}' + '\n',
      f'Quantidade de passos: {passos}')


Solução encontrada? True
 Estados visitados: [array([[2, 8, 3],
       [1, 6, 4],
       [7, 0, 5]]), array([[2, 8, 3],
       [1, 0, 4],
       [7, 6, 5]]), array([[2, 8, 3],
       [1, 6, 4],
       [0, 7, 5]]), array([[2, 8, 3],
       [1, 6, 4],
       [7, 5, 0]]), array([[2, 0, 3],
       [1, 8, 4],
       [7, 6, 5]]), array([[2, 8, 3],
       [0, 1, 4],
       [7, 6, 5]]), array([[2, 8, 3],
       [1, 4, 0],
       [7, 6, 5]]), array([[2, 8, 3],
       [0, 6, 4],
       [1, 7, 5]]), array([[2, 8, 3],
       [1, 6, 0],
       [7, 5, 4]]), array([[0, 2, 3],
       [1, 8, 4],
       [7, 6, 5]]), array([[2, 3, 0],
       [1, 8, 4],
       [7, 6, 5]]), array([[0, 8, 3],
       [2, 1, 4],
       [7, 6, 5]]), array([[2, 8, 3],
       [7, 1, 4],
       [0, 6, 5]]), array([[2, 8, 0],
       [1, 4, 3],
       [7, 6, 5]]), array([[2, 8, 3],
       [1, 4, 5],
       [7, 6, 0]]), array([[0, 8, 3],
       [2, 6, 4],
       [1, 7, 5]]), array([[2, 8, 3],
       [6, 0, 4],
       [1, 7, 5]]), ar

### Usando busca em profundidade (DFS):

<img src='https://github.com/alvaromfcunha-c210/aula5/blob/master/imagens/3.gif?raw=1' alt='3' width='300' height='300'>

In [4]:
#Essa classe representa uma pilha, que é uma estrutura de dados que segue o
#princípio "último a entrar, primeiro a sair" (Last-In-First-Out, ou LIFO)
from queue import LifoQueue

class DepthFirstSearch:

  #Inicializando o problema
  def __init__(self, problema: SlidingPuzzle):
    self.problema = problema

  #verifica se um determinado estado já foi visitado, comparando-o com os estados
  #presentes na lista estados_visitados.
  def _verifica_visitados(self, estado, estados_visitados):
    for i in estados_visitados:
      if self.problema.verifica_estados(i, estado):
        return True
    return False

  def busca(self, inicio, fim):
    #O método busca realiza a busca em profundidade utilizando uma pilha (LifoQueue).
    #Ele começa inserindo o estado inicio na pilha.
    pilha = LifoQueue()
    pilha.put(inicio)

    solucao_encontrada = False
    #Armazenará uma lista de estados visitados
    estados_visitados = []
    cont_estados = 0

    while not pilha.empty():
      #Ele retira o estado mais recente da pilha usando pilha.get().
      #OBS: A pilha sempre retira o último inserido
      atual = pilha.get()

      #Adiciona o estado atual à lista de estados_visitados.
      estados_visitados.append(atual)

      #Verifica se o estado atual é igual ao estado fim usando o método verifica_estados do problema.
      #Se for igual, define solucao_encontrada como True e interrompe o loop.
      if self.problema.verifica_estados(atual, fim):
        solucao_encontrada = True
        break
      #Caso contrário, incrementa cont_estados.
      else:
        cont_estados += 1

        # print(f'Visitando estado: {cont_estados}')

        #Obtém os novos estados possíveis a partir do estado atual usando self.problema.expande_estados(atual).
        novos_estados = self.problema.expande_estados(atual)
        #Itera por esses novos estados e verifica se eles já foram visitados usando o método _verifica_visitados.
        #Se não foram visitados, coloca esses novos estados na pilha.
        for i in novos_estados:
          if not self._verifica_visitados(i, estados_visitados):
            pilha.put(i)

    return solucao_encontrada, estados_visitados, cont_estados

# **Ao contrário da busca em largura, a busca em profundidade explora um caminho o mais profundo possível antes de retroceder para explorar outras ramificações. Portanto, a solução encontrada pode não ser a mais curta em termos de número de movimentos, mas a primeira solução encontrada é garantida de ser uma das mais profundas possíveis.**

In [5]:
dfs = DepthFirstSearch(problema)

solucao, visitados, passos = bfs.busca(inicio, alvo)

print(f'Solução encontrada? {solucao}' + '\n',
      f'Estados visitados: {visitados}' + '\n',
      f'Quantidade de passos: {passos}')

Solução encontrada? True
 Estados visitados: [array([[2, 8, 3],
       [1, 6, 4],
       [7, 0, 5]]), array([[2, 8, 3],
       [1, 0, 4],
       [7, 6, 5]]), array([[2, 8, 3],
       [1, 6, 4],
       [0, 7, 5]]), array([[2, 8, 3],
       [1, 6, 4],
       [7, 5, 0]]), array([[2, 0, 3],
       [1, 8, 4],
       [7, 6, 5]]), array([[2, 8, 3],
       [0, 1, 4],
       [7, 6, 5]]), array([[2, 8, 3],
       [1, 4, 0],
       [7, 6, 5]]), array([[2, 8, 3],
       [0, 6, 4],
       [1, 7, 5]]), array([[2, 8, 3],
       [1, 6, 0],
       [7, 5, 4]]), array([[0, 2, 3],
       [1, 8, 4],
       [7, 6, 5]]), array([[2, 3, 0],
       [1, 8, 4],
       [7, 6, 5]]), array([[0, 8, 3],
       [2, 1, 4],
       [7, 6, 5]]), array([[2, 8, 3],
       [7, 1, 4],
       [0, 6, 5]]), array([[2, 8, 0],
       [1, 4, 3],
       [7, 6, 5]]), array([[2, 8, 3],
       [1, 4, 5],
       [7, 6, 0]]), array([[0, 8, 3],
       [2, 6, 4],
       [1, 7, 5]]), array([[2, 8, 3],
       [6, 0, 4],
       [1, 7, 5]]), ar